# Flickr30K Dataset Exploration

This notebook helps you explore the Flickr30K dataset:
- Load and examine the captions file
- Display sample captions with their images
- Verify data integrity
- Create utility functions for data access

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

## 2. Set Up Paths

In [ ]:
# Define paths
DATA_DIR = Path("data")
IMAGES_DIR = DATA_DIR / "images"
CAPTIONS_FILE = DATA_DIR / "results.csv"

# Verify paths exist
print("Checking data directories...")
print(f"Data directory exists: {DATA_DIR.exists()}")
print(f"Images directory exists: {IMAGES_DIR.exists()}")
print(f"Captions file exists: {CAPTIONS_FILE.exists()}")

if not CAPTIONS_FILE.exists():
    print("\n⚠️  Warning: Captions file not found!")
    print("Please ensure 'results.csv' is in the data/ directory.")
    print("You may need to download it from Kaggle or the original source.")

## 3. Load Captions Data

In [ ]:
# Load the captions CSV file
try:
    df = pd.read_csv(CAPTIONS_FILE, delimiter='|')
    print(f"✓ Successfully loaded captions file")
    print(f"  Total rows: {len(df):,}")
    print(f"  Columns: {list(df.columns)}")
except FileNotFoundError:
    print("❌ Captions file not found. Please download the dataset first.")
    df = None
except Exception as e:
    print(f"❌ Error loading captions: {e}")
    # Try alternative delimiter
    try:
        df = pd.read_csv(CAPTIONS_FILE)
        print(f"✓ Loaded with default delimiter")
        print(f"  Total rows: {len(df):,}")
    except:
        df = None

In [ ]:
# Display first few rows
if df is not None:
    print("\nFirst 5 rows of the dataset:")
    display(df.head())
    
    print("\nDataset info:")
    print(df.info())
    
    print("\nColumn names:")
    for col in df.columns:
        print(f"  - {col}")

## 4. Data Preprocessing

Clean column names and prepare the data for analysis.

In [ ]:
if df is not None:
    # Clean column names (remove leading/trailing spaces)
    df.columns = df.columns.str.strip()
    
    # Check common column name variations
    possible_image_cols = ['image_name', 'image', 'filename', 'image_id']
    possible_caption_cols = ['comment', 'caption', 'description']
    possible_number_cols = ['comment_number', 'caption_number', 'number']
    
    # Find the actual column names
    image_col = next((col for col in df.columns if col.lower() in possible_image_cols), df.columns[0])
    caption_col = next((col for col in df.columns if any(x in col.lower() for x in possible_caption_cols)), df.columns[-1])
    
    print(f"Using columns:")
    print(f"  Image column: '{image_col}'")
    print(f"  Caption column: '{caption_col}'")
    
    # Standardize column names
    if image_col != 'image_name':
        df.rename(columns={image_col: 'image_name'}, inplace=True)
    if caption_col != 'caption':
        df.rename(columns={caption_col: 'caption'}, inplace=True)
    
    # Remove any whitespace from captions
    if 'caption' in df.columns:
        df['caption'] = df['caption'].str.strip()
    
    print("\n✓ Data preprocessing complete")
    print(f"  Unique images: {df['image_name'].nunique():,}")
    print(f"  Total captions: {len(df):,}")
    print(f"  Average captions per image: {len(df) / df['image_name'].nunique():.2f}")

## 5. Display Sample Captions

In [ ]:
if df is not None:
    # Select a random image and show all its captions
    random_image = df['image_name'].sample(1).values[0]
    image_captions = df[df['image_name'] == random_image]
    
    print(f"Sample captions for image: {random_image}")
    print("=" * 80)
    for idx, row in image_captions.iterrows():
        caption_num = row.get('comment_number', idx)
        print(f"Caption {caption_num}: {row['caption']}")
    print("=" * 80)

## 6. Utility Functions

In [ ]:
def load_image(image_filename):
    """
    Load and return an image given its filename.
    
    Args:
        image_filename (str): Name of the image file (e.g., '1000092795.jpg')
    
    Returns:
        PIL.Image: Loaded image object, or None if file not found
    """
    image_path = IMAGES_DIR / image_filename
    
    if not image_path.exists():
        print(f"❌ Image not found: {image_filename}")
        return None
    
    try:
        img = Image.open(image_path)
        return img
    except Exception as e:
        print(f"❌ Error loading image {image_filename}: {e}")
        return None


def display_image(image_filename, show_captions=True, figsize=(10, 8)):
    """
    Display an image and optionally its captions.
    
    Args:
        image_filename (str): Name of the image file
        show_captions (bool): Whether to display captions below the image
        figsize (tuple): Figure size for matplotlib
    """
    img = load_image(image_filename)
    
    if img is None:
        return
    
    # Display image
    plt.figure(figsize=figsize)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Image: {image_filename}", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Display captions if requested
    if show_captions and df is not None:
        captions = df[df['image_name'] == image_filename]['caption'].values
        if len(captions) > 0:
            print(f"\nCaptions for {image_filename}:")
            print("=" * 80)
            for i, caption in enumerate(captions, 1):
                print(f"{i}. {caption}")
            print("=" * 80)
        else:
            print(f"\n⚠️  No captions found for {image_filename}")


def get_captions(image_filename):
    """
    Get all captions for a given image.
    
    Args:
        image_filename (str): Name of the image file
    
    Returns:
        list: List of captions for the image
    """
    if df is None:
        return []
    
    captions = df[df['image_name'] == image_filename]['caption'].values
    return captions.tolist()


def display_random_samples(n_samples=3, figsize=(15, 5)):
    """
    Display random samples from the dataset.
    
    Args:
        n_samples (int): Number of random samples to display
        figsize (tuple): Figure size for matplotlib
    """
    if df is None:
        print("❌ No captions data available")
        return
    
    # Get random unique images
    random_images = df['image_name'].drop_duplicates().sample(n_samples).values
    
    fig, axes = plt.subplots(1, n_samples, figsize=figsize)
    if n_samples == 1:
        axes = [axes]
    
    for idx, (ax, img_name) in enumerate(zip(axes, random_images)):
        img = load_image(img_name)
        if img is not None:
            ax.imshow(img)
            ax.axis('off')
            ax.set_title(f"Image {idx + 1}", fontsize=12, fontweight='bold')
        else:
            ax.text(0.5, 0.5, 'Image not found', ha='center', va='center')
            ax.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Display captions
    print("\nCaptions for displayed images:")
    print("=" * 100)
    for idx, img_name in enumerate(random_images, 1):
        captions = get_captions(img_name)
        print(f"\nImage {idx} ({img_name}):")
        for i, caption in enumerate(captions, 1):
            print(f"  {i}. {caption}")
    print("=" * 100)


print("✓ Utility functions defined:")
print("  - load_image(image_filename)")
print("  - display_image(image_filename, show_captions=True)")
print("  - get_captions(image_filename)")
print("  - display_random_samples(n_samples=3)")

## 7. Test the Functions

Let's test our utility functions with sample images.

In [ ]:
# Display 3 random samples
display_random_samples(n_samples=3)

In [ ]:
# Display a specific image with captions
if df is not None:
    sample_image = df['image_name'].iloc[0]
    display_image(sample_image, show_captions=True)

## 8. Dataset Statistics

In [ ]:
if df is not None:
    print("Dataset Statistics:")
    print("=" * 60)
    
    # Basic stats
    n_unique_images = df['image_name'].nunique()
    n_total_captions = len(df)
    avg_captions = n_total_captions / n_unique_images
    
    print(f"Total unique images: {n_unique_images:,}")
    print(f"Total captions: {n_total_captions:,}")
    print(f"Average captions per image: {avg_captions:.2f}")
    
    # Caption length statistics
    df['caption_length'] = df['caption'].str.split().str.len()
    
    print(f"\nCaption Length Statistics (words):")
    print(f"  Mean: {df['caption_length'].mean():.2f}")
    print(f"  Median: {df['caption_length'].median():.0f}")
    print(f"  Min: {df['caption_length'].min()}")
    print(f"  Max: {df['caption_length'].max()}")
    print(f"  Std: {df['caption_length'].std():.2f}")
    
    # Check for actual images
    image_files = list(IMAGES_DIR.glob("*.jpg"))
    print(f"\nActual image files in {IMAGES_DIR}:")
    print(f"  Count: {len(image_files):,}")
    
    if len(image_files) > 0:
        # Check if caption images exist in folder
        sample_images = df['image_name'].head(100).values
        existing = sum(1 for img in sample_images if (IMAGES_DIR / img).exists())
        print(f"  Sample verification: {existing}/100 images from captions file found")
    
    print("=" * 60)

## 9. Visualize Caption Length Distribution

In [ ]:
if df is not None and 'caption_length' in df.columns:
    plt.figure(figsize=(12, 5))
    
    # Histogram
    plt.subplot(1, 2, 1)
    plt.hist(df['caption_length'], bins=50, edgecolor='black', alpha=0.7)
    plt.xlabel('Caption Length (words)', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.title('Distribution of Caption Lengths', fontsize=14, fontweight='bold')
    plt.grid(axis='y', alpha=0.3)
    
    # Box plot
    plt.subplot(1, 2, 2)
    plt.boxplot(df['caption_length'], vert=True)
    plt.ylabel('Caption Length (words)', fontsize=12)
    plt.title('Caption Length Box Plot', fontsize=14, fontweight='bold')
    plt.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 10. Sample Captions Analysis

In [ ]:
if df is not None:
    print("Sample of shortest captions:")
    print("=" * 80)
    shortest = df.nsmallest(5, 'caption_length')[['image_name', 'caption', 'caption_length']]
    for idx, row in shortest.iterrows():
        print(f"{row['caption']} ({row['caption_length']} words) - {row['image_name']}")
    
    print("\n" + "=" * 80)
    print("Sample of longest captions:")
    print("=" * 80)
    longest = df.nlargest(5, 'caption_length')[['image_name', 'caption', 'caption_length']]
    for idx, row in longest.iterrows():
        print(f"{row['caption']} ({row['caption_length']} words) - {row['image_name']}")
    print("=" * 80)

## 11. Summary and Next Steps

### What We've Done:
1. ✓ Loaded the Flickr30K captions dataset
2. ✓ Explored the dataset structure and statistics
3. ✓ Created utility functions for loading images and captions
4. ✓ Visualized sample images with their captions
5. ✓ Analyzed caption length distributions

### Next Steps for Hybrid Multimodal Retrieval:
1. **Preprocessing Pipeline**: Create data loaders for training
2. **Text Encoding**: Implement or use pre-trained text encoders (BERT, etc.)
3. **Image Encoding**: Implement or use pre-trained image encoders (ResNet, ViT, etc.)
4. **Embedding Space**: Design the joint embedding space
5. **Retrieval System**: Implement similarity search (cosine, etc.)
6. **Evaluation Metrics**: Set up Recall@K, MRR, etc.

### Useful Functions Created:
- `load_image(filename)`: Load an image by filename
- `display_image(filename, show_captions)`: Display image with captions
- `get_captions(filename)`: Get all captions for an image
- `display_random_samples(n)`: Show random image-caption pairs

In [ ]:
print("🎉 Dataset exploration complete!")
print("\nYou can now proceed with building your hybrid multimodal retrieval system.")
print("\nKey variables available:")
print(f"  - df: DataFrame with {len(df) if df is not None else 0:,} captions")
print(f"  - DATA_DIR: {DATA_DIR}")
print(f"  - IMAGES_DIR: {IMAGES_DIR}")
print(f"  - CAPTIONS_FILE: {CAPTIONS_FILE}")